### Import des bibliothéques
---

In [9]:
import pandas as pd
import numpy as np


from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import RandomForestClassifier, BaggingClassifier, GradientBoostingClassifier, StackingClassifier
from sklearn.metrics import f1_score, confusion_matrix, accuracy_score, precision_score, recall_score,confusion_matrix, classification_report, roc_curve, auc

###  Chargement des données
---

In [10]:
df = pd.read_csv("../data/conversion_data_train.csv", encoding="latin-1")
df.head()

,country,age,new_user,source,total_pages_visited,converted
0,China,22,1,Direct,2,0
1,UK,21,1,Ads,3,0
2,Germany,20,0,Seo,14,1
3,US,23,1,Seo,3,0
4,US,28,1,Direct,3,0


### Reduction du jeu de Données
---

In [11]:
# Le dataset est important, création d'un échantillon
df_sample = df.sample(10000)

### Modéle RandomForestClassifier 
---

In [12]:
# Liste des features et de la variable cible
features_list = ['total_pages_visited', 'age', 'new_user', 'source', 'country']
target_variable = 'converted'

# Séparer les variables indépendantes (X) et la variable cible (Y)
X = df_sample[features_list]
Y = df_sample[target_variable]

# Diviser le jeu de données en ensembles d'entraînement (80%) et de test (20%)
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42, stratify=Y)

# Pipeline pour les colonnes numériques : Imputation + Standardisation
numerical_columns = ['age', 'new_user', 'total_pages_visited']
numerical_pipeline = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='mean')),  # Remplacer les valeurs manquantes par la moyenne
    ('standardization', StandardScaler())  # Standardisation des données numériques
])

# Pipeline pour les colonnes catégorielles : Imputation + Encodage One-Hot
categorical_columns = ['country', 'source']
categorical_pipeline = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),  # Remplacer les valeurs manquantes par la modalité la plus fréquente
    ('encoder', OneHotEncoder(drop='first'))  # Encodage One-Hot des variables catégorielles
])

# Appliquer les pipelines numériques et catégoriels sur les colonnes spécifiées
feature_encoder = ColumnTransformer(transformers=[
    ('num', numerical_pipeline, numerical_columns),
    ('cat', categorical_pipeline, categorical_columns)
])

# Appliquer les transformations sur les données d'entraînement et de test
X_train = feature_encoder.fit_transform(X_train)
X_test = feature_encoder.transform(X_test)

# Optimisation des hyperparamètres avec GridSearchCV 
param_grid = {
    'n_estimators': [100, 200],
    'max_depth': [10, 20, None],
    'min_samples_split': [2, 5],
    'min_samples_leaf': [1, 2],
}

# Initialisation du Modéle 
rf_clf = RandomForestClassifier(random_state=42)
grid_search = GridSearchCV(estimator=rf_clf, param_grid=param_grid, cv=3, n_jobs=-1, verbose=2)
grid_search.fit(X_train, Y_train)

# Meilleur modèle après recherche d'hyperparamètres
best_rf_clf = grid_search.best_estimator_

# # Entraînement du modèle optimisé
# best_rf_clf.fit(X_train, Y_train)

# Prédictions
Y_train_pred = best_rf_clf.predict(X_train)
Y_test_pred = best_rf_clf.predict(X_test)

# Métriques 
print("--- Évaluation des performances du modèle --- \n")

# Train
print(" --- Train ---\n")
print(f"Accuracy  : {accuracy_score(Y_train, Y_train_pred):.4f}")
print(f"Precision : {precision_score(Y_train, Y_train_pred):.4f}")
print(f"Recall    : {recall_score(Y_train, Y_train_pred):.4f}")
print(f"F1 Score  : {f1_score(Y_train, Y_train_pred):.4f}\n")

# Test
print("--- Test ---\n ")
print(f"Accuracy  : {accuracy_score(Y_test, Y_test_pred):.4f}")
print(f"Precision : {precision_score(Y_test, Y_test_pred):.4f}")
print(f"Recall    : {recall_score(Y_test, Y_test_pred):.4f}")
print(f"F1 Score  : {f1_score(Y_test, Y_test_pred):.4f}")





Fitting 3 folds for each of 24 candidates, totalling 72 fits
--- Évaluation des performances du modèle --- 

 --- Train ---

Accuracy  : 0.9890
Precision : 0.9369
Recall    : 0.7376
F1 Score  : 0.8254

--- Test ---
 
Accuracy  : 0.9830
Precision : 0.8214
Recall    : 0.6571
F1 Score  : 0.7302


### Sauvegarde des résultats
---

In [13]:
# Création du DataFrame pour la première fois
scores_df = pd.DataFrame(columns=["model", "f1", "recall", "accuracy"])
new_rows = [{"model": "best_rf_clf", "f1": f1_score(Y_test, Y_test_pred), "recall": recall_score(Y_test, Y_test_pred), "accuracy":accuracy_score(Y_test, Y_test_pred)}]

scores_df = pd.concat([scores_df, pd.DataFrame(new_rows)], ignore_index=True)
scores_df.to_csv("../data/scores_models.csv", mode="a", header=False, index=False)

C:\Users\natha\AppData\Local\Temp\ipykernel_27264\145451096.py:5: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  scores_df = pd.concat([scores_df, pd.DataFrame(new_rows)], ignore_index=True)


### Prédictions finales
---

In [14]:
# toutes les données d'entrainement réunis
X = np.append(X_train, X_test, axis=0)
Y = np.append(Y_train, Y_test)

# reentrainement du modele
rf_clf.fit(X, Y)

# charger les données sans la target
data_without_labels = pd.read_csv('..\data\conversion_data_test.csv')

# selection des variables utilisées dans notre modéle
X_without_labels = data_without_labels[features_list]

# application du meme pretraitement que pr les données entrainement
X_without_labels = feature_encoder.transform(X_without_labels)

# Predictions
Y_prediction = rf_clf.predict(X_without_labels)

# sauvegarder fichier
pd.DataFrame({'converted' : Y_prediction}).to_csv('../resultats/conversion_data_test_predictions_nath_rf.csv', index=False)

<>:9: SyntaxWarning: invalid escape sequence '\d'
<>:9: SyntaxWarning: invalid escape sequence '\d'
C:\Users\natha\AppData\Local\Temp\ipykernel_27264\1941615342.py:9: SyntaxWarning: invalid escape sequence '\d'
  data_without_labels = pd.read_csv('..\data\conversion_data_test.csv')
